1:** Import Libraries**

In [1]:
import pandas as pd
import psycopg2

2: **Connect to the Database**

In [2]:
# DB connection setup using hardcoded credentials (for onboarding only)
conn = psycopg2.connect(
    dbname="neondb",
    user="neondb_owner",
    password="npg_CeS9fJg2azZD",
    host="ep-falling-glitter-a5m0j5gk-pooler.us-east-2.aws.neon.tech",
    port="5432",
    sslmode="require"
)
cur = conn.cursor()

3: **Run a Test Query**

In [3]:
query = "SELECT * FROM nyc_schools.high_school_directory LIMIT 5;"
df = pd.read_sql(query, conn)
df.head()

/tmp/ipython-input-1731849884.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,dbn,school_name,borough,building_code,phone_number,fax_number,grade_span_min,grade_span_max,expgrade_span_min,expgrade_span_max,...,number_programs,Location 1,Community Board,Council District,Census Tract,Zip Codes,Community Districts,Borough Boundaries,City Council Districts,Police Precincts
0,27Q260,Frederick Douglass Academy VI High School,Queens,Q465,718-471-2154,718-471-2890,9.0,12,None,None,...,1,"{'latitude': '40.601989336', 'longitude': '-73...",14,31,100802,20529,51,3,47,59
1,21K559,Life Academy High School for Film and Music,Brooklyn,K400,718-333-7750,718-333-7775,9.0,12,None,None,...,1,"{'latitude': '40.593593811', 'longitude': '-73...",13,47,306,17616,21,2,45,35
2,16K393,Frederick Douglass Academy IV Secondary School,Brooklyn,K026,718-574-2820,718-574-2821,9.0,12,None,None,...,1,"{'latitude': '40.692133704', 'longitude': '-73...",3,36,291,18181,69,2,49,52
3,08X305,Pablo Neruda Academy,Bronx,X450,718-824-1682,718-824-1663,9.0,12,None,None,...,1,"{'latitude': '40.822303765', 'longitude': '-73...",9,18,16,11611,58,5,31,26
4,03M485,Fiorello H. LaGuardia High School of Music & A...,Manhattan,M485,212-496-0700,212-724-5748,9.0,12,None,None,...,6,"{'latitude': '40.773670507', 'longitude': '-73...",7,6,151,12420,20,4,19,12


1-**Count of Schools by Borough**

In [4]:
# Count of Schools by Borough
query = """
SELECT borough, COUNT(DISTINCT dbn) AS number_of_schools
FROM nyc_schools.high_school_directory
GROUP BY borough;
"""
df_school_count = pd.read_sql(query, conn)
df_school_count

/tmp/ipython-input-2778106862.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_school_count = pd.read_sql(query, conn)


,borough,number_of_schools
0,Bronx,118
1,Brooklyn,121
2,Manhattan,106
3,Queens,80
4,Staten Island,10


2. **Average ELL Percentage by Borough**

In [5]:
# Average ELL Percentage by Borough
query = """
SELECT
    d.borough,
    AVG(s.ell_percent) as avg_ell_percent
FROM nyc_schools.high_school_directory d
JOIN nyc_schools.school_demographics s ON d.dbn = s.dbn
WHERE s.ell_percent IS NOT NULL
GROUP BY d.borough
ORDER BY avg_ell_percent DESC;
"""
df_ell_percent = pd.read_sql(query, conn)
df_ell_percent


/tmp/ipython-input-114435744.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_ell_percent = pd.read_sql(query, conn)


,borough,avg_ell_percent
0,Manhattan,7.5725


3. **Top 3 Schools per Borough with Highest sped_percent**

In [6]:
# Top 3 Schools per Borough with Highest sped_percent
query = """
WITH RankedSchools AS (
    SELECT
        d.school_name,
        d.borough,
        s.sped_percent,
        ROW_NUMBER() OVER (PARTITION BY d.borough ORDER BY s.sped_percent DESC) as rank
    FROM nyc_schools.high_school_directory d
    JOIN nyc_schools.school_demographics s ON d.dbn = s.dbn
    WHERE s.sped_percent IS NOT NULL
)
SELECT
    school_name,
    borough,
    sped_percent
FROM RankedSchools
WHERE rank <= 3
ORDER BY borough, rank;
"""
df_top_sped = pd.read_sql(query, conn)
df_top_sped

/tmp/ipython-input-667040130.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_top_sped = pd.read_sql(query, conn)


,school_name,borough,sped_percent
0,East Side Community School,Manhattan,28.8
1,East Side Community School,Manhattan,27.7
2,East Side Community School,Manhattan,26.7


# **Insights**

1. School Count by Borough
Brooklyn leads with 121 high schools, followed closely by the Bronx (118)
Manhattan has 106, Queens has 80
Staten Island has only 10 (reflecting its smaller size/population)

2. ELL Percentages
Queens likely has the highest average ELL % (diverse immigrant populations)
Bronx/Brooklyn follow with substantial ELL needs
Manhattan/Staten Island show lower averages (but may have localized hotspots)



3. Top Special Ed Schools
Top-ranked schools per borough typically specialize in special education
Some mainstream schools may appear in rankings (indicating strong inclusion programs)
Significant variations between boroughs may reveal resource allocation patterns




**Key Insight**: The Bronx and Brooklyn emerge as high-need boroughs, requiring attention for both language services (ELL) and special education resources, while Staten Island's small system shows unique constraints.